In [74]:
import numpy as np
import pandas as pd
import requests
from api import key

In [75]:
stocks = pd.read_csv('sp500Stocks.csv', names=['Ticker'])
stocks

,Ticker
0,MMM
1,ABT
2,ABBV
3,ABMD
4,ACN
...,...
500,YUM
501,ZBRA
502,ZBH
503,ZION


In [76]:
# Example of the data we'll retrieve

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={key}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2373100866388,
 'week52high': 161.32,
 'week52low': 107.49,
 'week52highSplitAdjustOnly': 160.8,
 'week52lowSplitAdjustOnly': 108.64,
 'week52change': 0.2465578433870619,
 'sharesOutstanding': 16723001443,
 'float': 0,
 'avg10Volume': 76847876,
 'avg30Volume': 89629286,
 'day200MovingAvg': 140.4,
 'day50MovingAvg': 155.3,
 'employees': 151869,
 'ttmEPS': 5.3,
 'ttmDividendRate': 0.876229328783868,
 'dividendYield': 0.00599227612184954,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-29',
 'nextEarningsDate': '2021-10-24',
 'peRatio': 28.22511136976654,
 'beta': 0,
 'maxChangePercent': 54.3421827967796,
 'year5ChangePercent': 4.580829101027147,
 'year2ChangePercent': 1.708776782499868,
 'year1ChangePercent': 0.2653648805468941,
 'ytdChangePercent': 0.08301575841946232,
 'month6ChangePercent': 0.1873594929622975,
 'month3ChangePercent': 0.05039513816474114,
 'month1ChangePercent': -0.04034178208153601,
 'day30ChangePercent': -0.069059522309

In [77]:
# Divide the stock lists in chunks of 100 (since iex allows batch api calls - max 100 symbols)
# This line of code was provided by https://stackoverflow.com/questions/9671224/split-a-python-list-into-other-sublists-i-e-smaller-lists
stocksChunks = [stocks['Ticker'][x:x+100] for x in range(0, len(stocks['Ticker']), 100)]

# Create a list of chunks of stocks (in string form delimited by ',') 
stocksBatchList = []
for i in range(0, len(stocksChunks)):
    stocksBatchList.append(','.join(stocksChunks[i]))
               
stocksBatchList[0]

'MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,AOS,APA,AAPL,AMAT,APTV,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,COG,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY'

In [78]:
# Batch api call to IEX
def getBatchData(symbols, data_types):
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types={data_types}&token={key}'
    return requests.get(api_url).json()

# getBatchData('AAPL,MMM','price,stats')

In [79]:
# Create the df in which we'll store the data retrieved from the api call

hqmColumns = ['Ticker', 'Price',
              '1Y PriceReturn', '1Y ReturnPercentile',
              '6M PriceReturn', '6M ReturnPercentile',
              '3M PriceReturn', '3M ReturnPercentile',
              '1M PriceReturn', '1M ReturnPercentile']

hqmDf = pd.DataFrame(columns = batchColumns)   

hqmDf

,Ticker,Price,1Y PriceReturn,1Y ReturnPercentile,6M PriceReturn,6M ReturnPercentile,3M PriceReturn,3M ReturnPercentile,1M PriceReturn,1M ReturnPercentile


In [80]:
# Populate the High Quality Momentum df with stats info and stock price

for stocksBatch in stocksBatchList:
    data = getBatchData(stocksBatch,'price,stats')
    
    for symbol in stocksBatch.split(','):
        
        hqmDf = hqmDf.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    None,
                    data[symbol]['stats']['month6ChangePercent'],
                    None,
                    data[symbol]['stats']['month3ChangePercent'],
                    None,
                    data[symbol]['stats']['month1ChangePercent'],
                    None
                ],
                index=batchColumns),
            ignore_index = True
        )
        
hqmDf

,Ticker,Price,1Y PriceReturn,1Y ReturnPercentile,6M PriceReturn,6M ReturnPercentile,3M PriceReturn,3M ReturnPercentile,1M PriceReturn,1M ReturnPercentile
0,MMM,186.51,0.168736,None,-0.063216,None,-0.072585,None,-0.074670,None
1,ABT,123.95,0.154229,None,-0.015535,None,0.024075,None,-0.043073,None
2,ABBV,109.07,0.292814,None,0.038240,None,-0.041290,None,-0.104403,None
3,ABMD,336.78,0.219440,None,0.080393,None,0.053748,None,-0.084622,None
4,ACN,333.19,0.473357,None,0.169858,None,0.096715,None,-0.037577,None
...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.06,0.390552,None,0.138940,None,0.069920,None,-0.068207,None
501,ZBRA,549.01,1.050763,None,0.127724,None,-0.020083,None,-0.108310,None
502,ZBH,149.52,0.132005,None,-0.067811,None,-0.072356,None,-0.003262,None
503,ZION,63.95,1.277560,None,0.204158,None,0.219409,None,0.090032,None


In [81]:
from scipy.stats import percentileofscore as score

# Calculate momentum percentiles

timePeriods = [ '1Y', '6M','3M', '1M'] 

for row in hqmDf.index:
    
    for timePeriod in timePeriods:
        
        changeCol = f'{timePeriod} PriceReturn'
        percentileCol = f'{timePeriod} ReturnPercentile'
        
        hqmDf.loc[row, percentileCol] = score(hqmDf[changeCol], batchDf.loc[row, changeCol])
        
hqmDf

,Ticker,Price,1Y PriceReturn,1Y ReturnPercentile,6M PriceReturn,6M ReturnPercentile,3M PriceReturn,3M ReturnPercentile,1M PriceReturn,1M ReturnPercentile
0,MMM,186.51,0.168736,24.356436,-0.063216,15.049505,-0.072585,14.257426,-0.074670,18.415842
1,ABT,123.95,0.154229,25.346535,-0.015535,25.544554,0.024075,56.039604,-0.043073,41.386139
2,ABBV,109.07,0.292814,37.821782,0.038240,40.792079,-0.041290,26.732673,-0.104403,5.544554
3,ABMD,336.78,0.219440,31.089109,0.080393,58.019802,0.053748,72.277228,-0.084622,19.009901
4,ACN,333.19,0.473357,62.772277,0.169858,72.277228,0.096715,87.128713,-0.037577,49.108911
...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.06,0.390552,47.524752,0.138940,68.712871,0.069920,73.465347,-0.068207,22.178218
501,ZBRA,549.01,1.050763,90.891089,0.127724,60.990099,-0.020083,45.940594,-0.108310,9.108911
502,ZBH,149.52,0.132005,18.019802,-0.067811,15.049505,-0.072356,14.059406,-0.003262,74.257426
503,ZION,63.95,1.277560,95.247525,0.204158,68.316832,0.219409,96.237624,0.090032,92.079208


In [82]:
# Calculate the HQM score ~ it is the mean of the 4 momentum percentile scores
from statistics import mean

for row in hqmDf.index:
    
    momentumPercentiles = []
    
    for timePeriod in timePeriods:
        momentumPercentiles.append(hqmDf.loc[row, f'{timePeriod} ReturnPercentile'])
    
    # Calculate and saves the HQM score in a new column
    hqmDf.loc[row, 'HQMScore'] = mean(momentumPercentiles)

hqmDf

,Ticker,Price,1Y PriceReturn,1Y ReturnPercentile,6M PriceReturn,6M ReturnPercentile,3M PriceReturn,3M ReturnPercentile,1M PriceReturn,1M ReturnPercentile,HQMScore
0,MMM,186.51,0.168736,24.356436,-0.063216,15.049505,-0.072585,14.257426,-0.074670,18.415842,18.019802
1,ABT,123.95,0.154229,25.346535,-0.015535,25.544554,0.024075,56.039604,-0.043073,41.386139,37.079208
2,ABBV,109.07,0.292814,37.821782,0.038240,40.792079,-0.041290,26.732673,-0.104403,5.544554,27.722772
3,ABMD,336.78,0.219440,31.089109,0.080393,58.019802,0.053748,72.277228,-0.084622,19.009901,45.099010
4,ACN,333.19,0.473357,62.772277,0.169858,72.277228,0.096715,87.128713,-0.037577,49.108911,67.821782
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.06,0.390552,47.524752,0.138940,68.712871,0.069920,73.465347,-0.068207,22.178218,52.970297
501,ZBRA,549.01,1.050763,90.891089,0.127724,60.990099,-0.020083,45.940594,-0.108310,9.108911,51.732673
502,ZBH,149.52,0.132005,18.019802,-0.067811,15.049505,-0.072356,14.059406,-0.003262,74.257426,30.346535
503,ZION,63.95,1.277560,95.247525,0.204158,68.316832,0.219409,96.237624,0.090032,92.079208,87.970297


In [83]:
# Select the first 50 momentum stocks and save the relevant data in a new df

hqmDf.sort_values('HQMScore', ascending=False, inplace = True)

final_df = hqmDf[:50][['Ticker','Price','HQMScore']]

final_df.reset_index(inplace=True, drop=True)

final_df.head()

,Ticker,Price,HQMScore
0,MCHP,156.06,99.702970
1,DVN,36.92,98.712871
2,SIVB,688.21,96.633663
3,PWR,120.95,95.742574
4,MRNA,381.10,93.415842


In [84]:
import math

# Claculate the amount of shares to buy based on a portfolio size

PORTFOLIO_SIZE = 1000000

positio_size = float(PORTFOLIO_SIZE)/len(final_df.index)

for i in final_df.index:
    # Calculate and saves the number of shares to buy in a new column
    final_df.loc[i, "SharesToBuy"] = math.floor(positio_size/final_df.loc[i,"Price"])

final_df

,Ticker,Price,HQMScore,SharesToBuy
0,MCHP,156.06,99.702970,128.0
1,DVN,36.92,98.712871,541.0
2,SIVB,688.21,96.633663,29.0
3,PWR,120.95,95.742574,165.0
4,MRNA,381.10,93.415842,52.0
5,CZR,116.50,92.920792,171.0
6,IPG,39.20,92.772277,510.0
7,COP,68.23,92.722772,293.0
8,CF,58.54,91.683168,341.0
9,MOS,36.64,90.247525,545.0


In [85]:
# Save the final df to an excel file

final_df.to_excel("raccomendedTrades.xlsx", index=False)